# Aggregation examples

Working through the [https://www.elastic.co/guide/en/elasticsearch/guide/current/aggregations.html](Aggregations) chapter of *[https://www.elastic.co/guide/en/elasticsearch/guide/current/index.html](ElasticSearch: The Definitive Guide)*

In [3]:
import elasticsearch
from elasticsearch import helpers
import json

def p(doc, max_lines=25):
    lines = json.dumps(doc, indent=2, sort_keys=True).split('\n')
    if not max_lines or len(lines) <= max_lines:
        print('\n'.join(lines))
    else:
        print('\n'.join(lines[:max_lines/2]) + '\n...\n' + '\n'.join(lines[-max_lines/2:]))

In [6]:
es = elasticsearch.Elasticsearch(hosts=[{'host': 'localhost', 'port': 9200}])

In [7]:
helpers.bulk(client=es, index='cars', doc_type='transactions', actions=[
        { "price" : 10000, "color" : "red", "make" : "honda", "sold" : "2014-10-28" },
        { "price" : 20000, "color" : "red", "make" : "honda", "sold" : "2014-11-05" },
        { "price" : 30000, "color" : "green", "make" : "ford", "sold" : "2014-05-18" },
        { "price" : 15000, "color" : "blue", "make" : "toyota", "sold" : "2014-07-02" },
        { "price" : 12000, "color" : "green", "make" : "toyota", "sold" : "2014-08-19" },
        { "price" : 20000, "color" : "red", "make" : "honda", "sold" : "2014-11-05" },
        { "price" : 80000, "color" : "red", "make" : "bmw", "sold" : "2014-01-01" },
        { "price" : 25000, "color" : "blue", "make" : "ford", "sold" : "2014-02-12" },
    ])

(8, [])

In [9]:
p(es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "popular_colors": {
                "terms": {
                    "field": "color"
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "popular_colors": {
      "buckets": [
        {
          "doc_count": 4, 
          "key": "red"
        }, 
        {
          "doc_count": 2, 
          "key": "blue"
        }, 
        {
          "doc_count": 2, 
          "key": "green"
        }
      ], 
      "doc_count_error_upper_bound": 0, 
      "sum_other_doc_count": 0
    }
  }, 
  "hits": {
    "hits": [], 
    "max_score": 0.0, 
    "total": 8
  }, 
  "timed_out": false, 
  "took": 4
}


In [10]:
p(es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "popular_colors": {
                "terms": {
                    "field": "color"
                },
                "aggs": {
                    "avg_price": {
                            "avg": {
                                "field": "price"
                            }
                        }
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "popular_colors": {
      "buckets": [
        {
          "avg_price": {
            "value": 32500.0
          }, 
          "doc_count": 4, 
          "key": "red"
        }, 
        {
          "avg_price": {
            "value": 20000.0
          }, 
          "doc_count": 2, 
          "key": "blue"
        }, 
        {
          "avg_price": {
            "value": 21000.0
          }, 
          "doc_count": 2, 
          "key": "green"
        }
      ], 
      "doc_count_error_upper_bound": 0, 
      "sum_other_doc_count": 0
    }
  }, 
  "hits": {
    "hits": [], 
    "max_score": 0.0, 
    "total": 8
  }, 
  "timed_out": false, 
  "took": 20
}


In [11]:
p(es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "popular_colors": {
                "terms": {
                    "field": "color"
                },
                "aggs": {
                    "avg_price": {
                            "avg": {
                                "field": "price"
                            }
                        },
                        "make": {
                            "terms": {
                                "field": "make"
                            }
                        }
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "popular_colors": {
      "buckets": [
        {
          "avg_price": {
            "value": 32500.0
          }, 
          "doc_count": 4, 
          "key": "red", 
          "make": {
            "buckets": [
              {
                "doc_count": 3, 
                "key": "honda"
              }, 
              {
                "doc_count": 1, 
                "key": "bmw"
              }
            ], 
            "doc_count_error_upper_bound": 0, 
            "sum_other_doc_count": 0
          }
        }, 
        {
          "avg_price": {
            "value": 20000.0
          }, 
          "doc_count": 2, 
          "key": "blue", 
          "make": {
            "buckets": [
              {
                "doc_count": 1, 
                "key": "ford"
              }, 
              {
                "doc_count": 1, 
                "key": "toyota"
              

In [12]:
p(es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "popular_colors": {
                "terms": {
                    "field": "color"
                },
                "aggs": {
                    "avg_price": {
                            "avg": {
                                "field": "price"
                            }
                        },
                        "make": {
                            "terms": {
                                "field": "make"
                            },
                            "aggs": {
                                "min_price": { "min": { "field": "price" }},
                                "max_price": { "max": { "field": "price" }},
                            }
                        }
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "popular_colors": {
      "buckets": [
        {
          "avg_price": {
            "value": 32500.0
          }, 
          "doc_count": 4, 
          "key": "red", 
          "make": {
            "buckets": [
              {
                "doc_count": 3, 
                "key": "honda", 
                "max_price": {
                  "value": 20000.0
                }, 
                "min_price": {
                  "value": 10000.0
                }
              }, 
              {
                "doc_count": 1, 
                "key": "bmw", 
                "max_price": {
                  "value": 80000.0
                }, 
                "min_price": {
                  "value": 80000.0
                }
              }
            ], 
            "doc_count_error_upper_bound": 0, 
            "sum_other_doc_count": 0
          }
        }, 
        {
          "avg

In [13]:
es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "price": {
                "histogram": {
                    "field": "price",
                    "interval": 20000
                },
                "aggs": {
                    "revenue": {
                        "sum": {
                            "field": "price"
                        }
                    }
                }
            }
        }
    })

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'price': {u'buckets': [{u'doc_count': 3,
     u'key': 0,
     u'revenue': {u'value': 37000.0}},
    {u'doc_count': 4, u'key': 20000, u'revenue': {u'value': 95000.0}},
    {u'doc_count': 0, u'key': 40000, u'revenue': {u'value': 0.0}},
    {u'doc_count': 0, u'key': 60000, u'revenue': {u'value': 0.0}},
    {u'doc_count': 1, u'key': 80000, u'revenue': {u'value': 80000.0}}]}},
 u'hits': {u'hits': [], u'max_score': 0.0, u'total': 8},
 u'timed_out': False,
 u'took': 8}

In [15]:
p(es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "makes": {
                "terms": {
                    "field": "make",
                    "size": 10
                },
                "aggs": {
                    "stats": {
                        "extended_stats": {
                            "field": "price"
                        }
                    }
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "makes": {
      "buckets": [
        {
          "doc_count": 3, 
          "key": "honda", 
          "stats": {
            "avg": 16666.666666666668, 
            "count": 3, 
            "max": 20000.0, 
            "min": 10000.0, 
            "std_deviation": 4714.045207910315, 
            "std_deviation_bounds": {
              "lower": 7238.5762508460375, 
              "upper": 26094.757082487296
            }, 
            "sum": 50000.0, 
            "sum_of_squares": 900000000.0, 
            "variance": 22222222.22222221
          }
        }, 
        {
          "doc_count": 2, 
          "key": "ford", 
          "stats": {
            "avg": 27500.0, 
            "count": 2, 
            "max": 30000.0, 
            "min": 25000.0, 
            "std_deviation": 2500.0, 
            "std_deviation_bounds": {
              "lower": 22500.0, 
              "upper": 3250

In [17]:
p(es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "sales": {
                "date_histogram": {
                    "field": "sold",
                    "interval": "month",
                    "format": "yyyy-MM-dd",
                    "min_doc_count": 0,
                    "extended_bounds": {
                        "min": "2014-01-01",
                        "max": "2014-12-31"
                    }
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "sales": {
      "buckets": [
        {
          "doc_count": 1, 
          "key": 1388534400000, 
          "key_as_string": "2014-01-01"
        }, 
        {
          "doc_count": 1, 
          "key": 1391212800000, 
          "key_as_string": "2014-02-01"
        }, 
        {
          "doc_count": 0, 
          "key": 1393632000000, 
          "key_as_string": "2014-03-01"
        }, 
        {
          "doc_count": 0, 
          "key": 1396310400000, 
          "key_as_string": "2014-04-01"
        }, 
        {
          "doc_count": 1, 
          "key": 1398902400000, 
          "key_as_string": "2014-05-01"
        }, 
        {
          "doc_count": 0, 
          "key": 1401580800000, 
          "key_as_string": "2014-06-01"
        }, 
        {
          "doc_count": 1, 
          "key": 1404172800000, 
          "key_as_string": "2014-07-01"
        }, 
        {
    

In [19]:
p(es.search(index='cars', doc_type='transactions', body={
        "size": 0,
        "aggs": {
            "sales": {
                "date_histogram": {
                    "field": "sold",
                    "interval": "quarter",
                    "format": "yyyy-MM-dd",
                    "min_doc_count": 0,
                    "extended_bounds": {
                        "min": "2014-01-01",
                        "max": "2014-12-31"
                    }
                },
                "aggs": {
                    "revenue": { "sum": { "field": "price" } },
                    "revenue_by_make": {
                        "terms": {
                            "field": "make"
                        },
                        "aggs": {
                            "revenue": { "sum": { "field": "price" }}
                        }
                    }
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "sales": {
      "buckets": [
        {
          "doc_count": 2, 
          "key": 1388534400000, 
          "key_as_string": "2014-01-01", 
          "revenue": {
            "value": 105000.0
          }, 
          "revenue_by_make": {
            "buckets": [
              {
                "doc_count": 1, 
                "key": "bmw", 
                "revenue": {
                  "value": 80000.0
                }
              }, 
              {
                "doc_count": 1, 
                "key": "ford", 
                "revenue": {
                  "value": 25000.0
                }
              }
            ], 
            "doc_count_error_upper_bound": 0, 
            "sum_other_doc_count": 0
          }
        }, 
        {
          "doc_count": 1, 
          "key": 1396310400000, 
          "key_as_string": "2014-04-01", 
          "revenue": {
            "va

In [21]:
p(es.search(index='cars', doc_type='transactions', body={
        "query": {
            "match": {
                "make": "ford"
            }
        },
        "aggs": {
            "colors": {
                "terms": { "field": "color" }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "colors": {
      "buckets": [
        {
          "doc_count": 1, 
          "key": "blue"
        }, 
        {
          "doc_count": 1, 
          "key": "green"
        }
      ], 
      "doc_count_error_upper_bound": 0, 
      "sum_other_doc_count": 0
    }
  }, 
  "hits": {
    "hits": [
      {
        "_id": "AVg96eykbCBjQi1n-k6a", 
        "_index": "cars", 
        "_score": 1.4054651, 
        "_source": {
          "color": "green", 
          "make": "ford", 
          "price": 30000, 
          "sold": "2014-05-18"
        }, 
        "_type": "transactions"
      }, 
      {
        "_id": "AVg96eylbCBjQi1n-k6f", 
        "_index": "cars", 
        "_score": 1.0, 
        "_source": {
          "color": "blue", 
          "make": "ford", 
          "price": 25000, 
          "sold": "2014-02-12"
        }, 
        "_type": "transactions"
      }
    ], 
    "max_score"

In [22]:
p(es.search(index='cars', doc_type='transactions', body={
        "query": {
            "match": { "make": "ford" }
        },
        "aggs": {
            "single_avg_price": {
                "avg": { "field": "price" }
            },
            "all": {
                "global": {},
                "aggs": {
                    "avg_price": { "avg": { "field": "price" }}
                }
            }
        }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "all": {
      "avg_price": {
        "value": 26500.0
      }, 
      "doc_count": 8
    }, 
    "single_avg_price": {
      "value": 27500.0
    }
  }, 
  "hits": {
    "hits": [
      {
        "_id": "AVg96eykbCBjQi1n-k6a", 
        "_index": "cars", 
        "_score": 1.4054651, 
        "_source": {
          "color": "green", 
          "make": "ford", 
          "price": 30000, 
          "sold": "2014-05-18"
        }, 
        "_type": "transactions"
      }, 
      {
        "_id": "AVg96eylbCBjQi1n-k6f", 
        "_index": "cars", 
        "_score": 1.0, 
        "_source": {
          "color": "blue", 
          "make": "ford", 
          "price": 25000, 
          "sold": "2014-02-12"
        }, 
        "_type": "transactions"
      }
    ], 
    "max_score": 1.4054651, 
    "total": 2
  }, 
  "timed_out": false, 
  "took": 5
}


In [25]:
p(es.search(
        index='cars', doc_type='transactions', body={
            "size": 0,
            "query": {
                "constant_score": {
                    "filter": {
                        "range": {
                            "price": { "gt": 10000 }
                        }
                    }
                }
            },
            "aggs": {
                "avg_price": {
                    "avg": {"field": "price"}
                }
            }
    }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "avg_price": {
      "value": 28857.14285714286
    }
  }, 
  "hits": {
    "hits": [], 
    "max_score": 0.0, 
    "total": 7
  }, 
  "timed_out": false, 
  "took": 5
}


---

Parent-child relationships

---

In [27]:
es.indices.create(
    index='company', body={
        "mappings": {
            "branch": {},
            "employee": {
                "_parent": {
                    "type": "branch"
                }
            }
        }
    })

{u'acknowledged': True}

In [30]:
helpers.bulk(es, index='company', doc_type='branch', actions=[
        { "_id": "london",    "name": "London Westminster", "city": "London", "country": "UK" },
        { "_id": "liverpool", "name": "Liverpool Central", "city": "Liverpool", "country": "UK" },
        { "_id": "paris",     "name": u"Champs Élysées", "city": "Paris", "country": "France" },
    ])

(3, [])

In [31]:
es.index(
    index='company', doc_type='employee', id='1', parent='london', body={
      "name":  "Alice Smith",
      "dob":   "1970-10-24",
      "hobby": "hiking"
    })

{u'_id': u'1',
 u'_index': u'company',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'employee',
 u'_version': 1,
 u'created': True}

In [32]:
helpers.bulk(
    es, index='company', doc_type='employee', actions=[
        { "_id": 2, "_parent": "london", "name": "Mark Thomas", "dob": "1982-05-16", "hobby": "diving" },
        { "_id": 3, "_parent": "liverpool", "name": "Barry Smith", "dob": "1979-04-01", "hobby": "hiking" },
        { "_id": 4, "_parent": "paris", "name": "Adrien Grand", "dob": "1987-05-11", "hobby": "horses" },
    ]
)

(3, [])

In [34]:
p(es.search(
    index='company', doc_type='branch', body={
        "query": {
            "has_child": {
                "type": "employee",
                "query": {
                    "range": {
                        "dob": {
                            "gte": "1980-01-01"
                        }
                    }
                }
            }
        }
    }
), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "hits": {
    "hits": [
      {
        "_id": "paris", 
        "_index": "company", 
        "_score": 1.0, 
        "_source": {
          "city": "Paris", 
          "country": "France", 
          "name": "Champs \u00c9lys\u00e9es"
        }, 
        "_type": "branch"
      }, 
      {
        "_id": "london", 
        "_index": "company", 
        "_score": 1.0, 
        "_source": {
          "city": "London", 
          "country": "UK", 
          "name": "London Westminster"
        }, 
        "_type": "branch"
      }
    ], 
    "max_score": 1.0, 
    "total": 2
  }, 
  "timed_out": false, 
  "took": 10
}


In [36]:
p(
    es.search(
        index='company', doc_type='employee', body={
            "query": {
                "has_parent": {
                    "type": "branch",
                    "query": {
                        "match": {
                            "country": "UK"
                        }
                    }
                }
            }
        }
    ),
    None
)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "hits": {
    "hits": [
      {
        "_id": "3", 
        "_index": "company", 
        "_parent": "liverpool", 
        "_routing": "liverpool", 
        "_score": 1.0, 
        "_source": {
          "dob": "1979-04-01", 
          "hobby": "hiking", 
          "name": "Barry Smith"
        }, 
        "_type": "employee"
      }, 
      {
        "_id": "1", 
        "_index": "company", 
        "_parent": "london", 
        "_routing": "london", 
        "_score": 1.0, 
        "_source": {
          "dob": "1970-10-24", 
          "hobby": "hiking", 
          "name": "Alice Smith"
        }, 
        "_type": "employee"
      }, 
      {
        "_id": "2", 
        "_index": "company", 
        "_parent": "london", 
        "_routing": "london", 
        "_score": 1.0, 
        "_source": {
          "dob": "1982-05-16", 
          "hobby": "diving", 
          "name": "Mark Thomas"
        }, 
 

In [38]:
p(es.cluster.state(), None)

{
  "blocks": {}, 
  "cluster_name": "elasticsearch", 
  "master_node": "GAsmR5iBRj-DtwlTVPNIZA", 
  "metadata": {
    "cluster_uuid": "k1vyrKjGQQOdfwGYCDBPdQ", 
    "indices": {
      "blogs": {
        "aliases": [], 
        "mappings": {}, 
        "settings": {
          "index": {
            "creation_date": "1478273698129", 
            "number_of_replicas": "2", 
            "number_of_shards": "3", 
            "uuid": "JTNn6NNpROuaSNx3-AkUWw", 
            "version": {
              "created": "2040199"
            }
          }
        }, 
        "state": "open"
      }, 
      "cars": {
        "aliases": [], 
        "mappings": {
          "transactions": {
            "properties": {
              "color": {
                "type": "string"
              }, 
              "make": {
                "type": "string"
              }, 
              "price": {
                "type": "long"
              }, 
              "sold": {
                "format": "strict_date_op